#### Desnormalizando datos de nuestra base de datos SQL del delivery para juntar información relevante en un solo objeto JSON/TEXTO Optimizado para: Búsqueda semántica, contexto completo, embeddings

In [68]:
import os
import pandas as pd
from sqlalchemy import create_engine
from urllib.parse import quote_plus
from dotenv import load_dotenv

# Cargar variables de entorno
load_dotenv()

# === DATOS DE CONEXIÓN ===
DB_USER = os.getenv("DB_USER")
DB_PASS = quote_plus(os.getenv("DB_PASS", "")) # se codifica aquí
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")

# === ENGINE ===
engine = create_engine(
    f"mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}",
    pool_pre_ping=True
)

# === PRUEBA DE CONEXIÓN ===
with engine.connect() as conn:
    print("✅ Conexión exitosa a MySQL")

# === CARGAR TABLA ===
negocios = pd.read_sql("""
SELECT
  id,
  name,
  description,
  horario_apertura,
  horario_cierre,
  diasOperacion,
  direccion,
  created_at,
  updated_at,
  descripcion_detallada
FROM categories
""", engine)

negocios.head()

✅ Conexión exitosa a MySQL


,id,name,description,horario_apertura,horario_cierre,diasOperacion,direccion,created_at,updated_at,descripcion_detallada
0,31,TACOS LOS LICENCIADOS,Tacos de carne asada,0 days 12:02:14,0 days 16:24:56,Sábado a Domingo,"Calle 66, Ciudad 16",2025-12-23 23:13:39,2025-09-23 21:54:51,Negocio de comida que ofrece tacos de asada en...
1,33,02-PIZZA CENTRAL,"Pizzas, papas y hamburguesas",0 days 13:21:26,0 days 19:43:15,Lunes a Viernes,"Calle 96, Ciudad 50",2025-12-26 12:52:28,2025-09-23 21:57:17,None
2,34,DESAYUNOS GAHORY,Desayunos en general,0 days 09:03:39,0 days 17:33:48,Miércoles a Domingo,"Calle 65, Ciudad 43",2025-12-24 11:32:03,2025-09-23 21:58:13,None
3,35,02-EXPRESSO CENTRAL,"Cafetería, crepas y más",0 days 11:04:55,0 days 21:53:39,Lunes a Domingo,"Calle 15, Ciudad 33",2025-12-26 12:52:34,2025-09-23 21:59:43,None
4,36,"HAMBURGUESAS ""EL GÜERO""","Hamburguesas, papas, hot-dogs",0 days 16:24:01,0 days 14:21:40,Jueves a Domingo,"Calle 60, Ciudad 50",2025-12-23 23:13:58,2025-09-23 22:00:19,Negocio de comida rápida que ofrece hamburgues...


In [69]:
productos = pd.read_sql("""
SELECT
  id,
  name,
  description,
  price,
  image1,
  image2,
  image3,
  id_category
FROM products
""", engine)

productos.head()

,id,name,description,price,image1,image2,image3,id_category
0,38,ENCHILADAS(DESAYUNOS GAHORY),CAMPIRANO:VERDES:POLLO / HUEVO,70.0,https://firebasestorage.googleapis.com/v0/b/te...,https://firebasestorage.googleapis.com/v0/b/te...,https://firebasestorage.googleapis.com/v0/b/te...,34
1,39,ENCHILADAS NORTEÑA(DESAYUNOS GAHORY),ROJAS C/PECHUGA ASADA ENCEBOLLADA,85.0,https://firebasestorage.googleapis.com/v0/b/te...,https://firebasestorage.googleapis.com/v0/b/te...,https://firebasestorage.googleapis.com/v0/b/te...,34
2,40,ENCHILADAS SUIZO(DESAYUNOS GAHORY),EN SALSA CREMOSITA GRATINADAS EN QUESO MANCHEGO,95.0,https://firebasestorage.googleapis.com/v0/b/te...,https://firebasestorage.googleapis.com/v0/b/te...,https://firebasestorage.googleapis.com/v0/b/te...,34
3,41,ENCHILADAS DELICIA(DESAYUNOS GAHORY),DIVORCIADAS,85.0,https://firebasestorage.googleapis.com/v0/b/te...,https://firebasestorage.googleapis.com/v0/b/te...,https://firebasestorage.googleapis.com/v0/b/te...,34
4,42,ENCHILADAS PAISA(DESAYUNOS GAHORY),ENCHILADAS EN SALSA DE FRIJOL,85.0,https://firebasestorage.googleapis.com/v0/b/te...,https://firebasestorage.googleapis.com/v0/b/te...,https://firebasestorage.googleapis.com/v0/b/te...,34


##### La parte de desnormalizar (preparado el JSON para el RAG): 




In [70]:
# ============================================
# Prepara el JSON base desde SQL
# ============================================

docs = []

def clean_time(t):
    if pd.isna(t):
        return None

    if isinstance(t, pd.Timedelta):
        total_seconds = int(t.total_seconds())
        hours = total_seconds // 3600
        minutes = (total_seconds % 3600) // 60
        return f"{hours:02d}:{minutes:02d}"

    if isinstance(t, str):
        return t[:5]

    if hasattr(t, "strftime"):
        return t.strftime("%H:%M")

    return None


def clean_text(t):
    if pd.isna(t) or t is None or str(t).strip() == "":
        return None
    return str(t).strip()


for _, n in negocios.iterrows():
    prods = productos[productos.id_category == n.id]

    descripcion_corta = clean_text(n.description)
    descripcion_detallada = clean_text(n.descripcion_detallada)

    # Fallback inteligente para descripción
    if not descripcion_detallada and descripcion_corta:
        descripcion_detallada = f"Negocio de comida que ofrece {descripcion_corta.lower()}."

    nombre = clean_text(n["name"])



    docs.append({
        # ─────────────────────────────
        # IDENTIDAD
        # ─────────────────────────────
        "document_id": f"negocio_{n.id}",
        "tipo": "negocio",
        "nombre": nombre,

        # ─────────────────────────────
        # DESCRIPCIONES
        # ─────────────────────────────
        "descripcion_corta": descripcion_corta,
        "descripcion_detallada": descripcion_detallada,

        # ─────────────────────────────
        # UBICACIÓN
        # ─────────────────────────────
        "direccion": clean_text(n.direccion),

        # ─────────────────────────────
        # HORARIOS
        # ─────────────────────────────
        "horarios": {
            "dias": clean_text(n.diasOperacion),
            "apertura": clean_time(n.horario_apertura),
            "cierre": clean_time(n.horario_cierre)
        },

        # ─────────────────────────────
        # PRODUCTOS
        # ─────────────────────────────
        "productos": (
            prods[["name", "description", "price"]]
            .rename(columns={
                "name": "nombre",
                "description": "descripcion",
                "price": "precio"
            })
            .map(clean_text)        # ✅ CORRECTO
            .to_dict("records")
        )
    })

In [71]:
#PARA EXPORTAR EL JSON, EL PRIMERO,
#CON LOS DATOS ORIGINALES
import json

with open("datos.json", "w", encoding="utf-8") as f:
    json.dump(docs, f, ensure_ascii=False, indent=2)

print("✅ Dataset RAG generado")

✅ Dataset RAG generado


In [72]:
###############LEER IMPORTANTE#################
# SISTEMA diseñado para aprender de forma robusta.

# Cada corrida analiza toda la plataforma (cross-negocio)

# Guarda todo en labels_pendientes.json con evidencia

# Si activas AUTO_PROMOTE_LABEL=True, promueve solo si:

# aparece en muchos negocios

# tiene muchas menciones

# y no es un término basura

#Todo queda guardado en label_registry.json y se usa en corridas futuras
#######################
#¿QUE ES?
# ✅ un motor de conocimiento incremental
# ✅ con memoria persistente
# ✅ con reglas + evidencia
# ✅ listo para RAG / embeddings / búsqueda semántica

#######################################

# El fin de todo este bloque es convertir texto sucio de negocios 
# y productos en un sistema de clasificación que aprende solo, 
# recuerda lo aprendido y solo evoluciona cuando hay evidencia 
# real a nivel plataforma.

###########################
# Es un sistema de clasificación semántica con aprendizaje heurístico global 
# y gobernanza automática, usado como preprocesador para RAG
###########################
# Genera docs_rag.json (negocio + producto) robusto y auto-actualizable:
# - Reglas base (LABEL_RULES)
# - Conceptos canónicos (CANONICAL_CONCEPTS) -> tags/booleans
# - Aprendizaje GLOBAL cross-negocio: descubre nuevos términos/phrases
# - Registro persistente de labels/tags: label_registry.json
# - Pendientes con evidencia: labels_pendientes.json
#
# Ideal para: Qwen3 + embeddings + Chroma/FAISS + n8n

import json
import re
import unicodedata
from collections import Counter, defaultdict
from pathlib import Path
from datetime import datetime

# =========================
# Config
# =========================
INPUT_DATOS = "datos.json"
OUTPUT_DOCS_RAG = "docs_rag.json"

REGISTRY_PATH = "label_registry.json" 
#label_registry.json no se recalcula desde cero
# Contiene:
# -labels_oficiales: etiquetas confiables para UI, filtros, dashboards
# -auto_promoted: historial de etiquetas que el sistema promovió solo
# Sin esto, cada corrida sería “amnesia total”.
# Con esto, el sistema madura con el tiempo.


PENDING_PATH = "labels_pendientes.json"
# Sirve para:
# -auditar
# -ajustar umbrales
# -aprobar manualmente
# -entender qué está aprendiendo el sistema

TOP_TAGS_PER_BIZ = 24
TOP_CANDIDATES_PER_BIZ = 12

MAX_NGRAMS = 2  # 2 = bigramas, 3 = trigramas

# Umbrales de “aprendizaje global”
# (ajustar según tamaño de la plataforma)
MIN_BUSINESSES_FOR_TAG = 2        # para aceptar tags globales
MIN_BUSINESSES_FOR_LABEL = 5      # para proponer como label candidata (categoría)
AUTO_PROMOTE_LABEL = True         # si True, promueve labels cuando pasa umbral
AUTO_PROMOTE_MIN_BUSINESSES = 8   # negocios distintos
AUTO_PROMOTE_MIN_TOTAL_MENTIONS = 20  # menciones totales (tokens/ngrams)
AUTO_PROMOTE_MIN_LEN = 4          # mínimo largo para auto-promoción

# =========================
# Stopwords / ruido
# =========================
STOPWORDS = {
    "de","la","el","y","o","con","sin","para","por","en","a","al","del","los","las",
    "un","una","unos","unas","que","se","su","sus","tipo","ofrece","negocio","comida",
    "incluye","pieza","piezas","orden","ml","lt","l","litro","litros","media","medio",
    "grande","familiar","especial","clasica","clasico","clásico","clasica","sencilla",
    "sencillo","combo","paquete","promocion","promoción","envio","envío","gratis",
    "desde","hasta","precio","precios","aprox","aproximado","a","elegir"
}

GENERIC_MENU_WORDS = {
    "rico","rica","delicioso","deliciosa","sabroso","sabrosa","caliente","crujiente",
    "suave","casero","casera","artesanal","premium","favorito","favorita",
    "fresco","fresca","natural","hecho","hecha","especialidad","tradicional"
}

UNITS = {"kg","kilo","kilos","gr","g","ml","lt","l","litro","litros","pz","pza","pzas","pieza","piezas"}

# =========================
# Normalización
# =========================
def normalize(text: str) -> str:
    if not text:
        return ""
    text = text.lower().strip()
    text = unicodedata.normalize("NFD", text)
    text = "".join(ch for ch in text if unicodedata.category(ch) != "Mn")
    text = re.sub(r"[^a-z0-9\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

def tokenize(text: str):
    text = normalize(text)
    toks = []
    for t in text.split():
        if not t or t in STOPWORDS:
            continue
        if len(t) < 3:
            continue
        if t.isdigit():
            continue
        if t in UNITS:
            continue
        if t in GENERIC_MENU_WORDS:
            continue
        toks.append(t)
    return toks

def make_ngrams(tokens, n=2):
    return [" ".join(tokens[i:i+n]) for i in range(len(tokens)-n+1)]

def is_bad_term(term: str) -> bool:
    if not term:
        return True
    if any(ch.isdigit() for ch in term):
        return True
    parts = term.split()
    if len(parts) == 1:
        if len(term) < 3:
            return True
        if term in STOPWORDS or term in GENERIC_MENU_WORDS or term in UNITS:
            return True
    # Evita términos absurdamente genéricos
    if term in STOPWORDS or term in GENERIC_MENU_WORDS or term in UNITS:
        return True
    return False

# =========================
# Labels reglas (base)
# =========================
LABEL_RULES = {
    "tacos": ["taco", "tacos", "taqueria", "taquería", "pastor", "suadero", "bistec", "arrachera", "longaniza", "chistorra"],
    "pizza": ["pizza", "pepperoni", "hawai", "hawaiana", "margarita", "tokio", "suprema"],
    "hamburguesas": ["hamburguesa", "burger", "hotdog", "hot-dog", "hot dog", "papas", "salchipulpos", "salchicha", "boneless", "alitas"],
    "desayunos": ["desayuno", "chilaquiles", "huevos", "hotcakes", "omelette", "enchiladas", "molletes", "cafe de olla", "tamales", "atole"],
    "cafeteria": ["cafe", "café", "capuchino", "latte", "moka", "frappe", "chai", "matcha", "chocolate caliente", "crepa", "crepas"],
    "mariscos": ["mariscos", "coctel", "cóctel", "camaron", "camarón", "mojarra", "ceviche", "aguachile"],
    "antojitos": ["quesadilla", "quesadillas", "sopes", "sope", "gorditas", "tlacoyo", "tostada", "flautas", "gringas", "huarache", "chalupas"],
    "postres": ["pastel", "rebanada", "galletas", "waffles", "crepa", "nutella", "cajeta", "bombones", "granola", "flan", "gelatina", "churros", "helado"],
    "bebidas": ["refresco", "coca", "cocacola", "boing", "agua", "limonada", "jugo", "te", "té", "malteada", "horchata", "jamaica", "tamarindo"],
    "alcohol": ["vinateria", "vinatería", "cerveza", "six", "laton", "latón", "tequila", "mezcal", "whisky", "red label", "pulque", "michelada", "chelada"],
}

# =========================
# Conceptos canónicos (alias -> tag fuerte)
# =========================
CANONICAL_CONCEPTS = {
    "comida corrida": [
        "comida corrida", "cocina economica", "cocina económica",
        "fonda", "comedor", "menu del dia", "menú del día",
        "guisados", "comida casera", "corrida"
    ],
    "tacos de canasta": ["tacos de canasta", "tacos sudados", "tacos al vapor"],
    "barbacoa": ["barbacoa", "consome de barbacoa", "consomé de barbacoa"],
    "birria": ["birria", "consome", "consomé"],
    "carnitas": ["carnitas"],
    "cochinita pibil": ["cochinita", "pibil", "cochinita pibil"],
}

def apply_canonical_tags(full_text: str):
    norm = normalize(full_text)
    tags = set()
    for canon, variants in CANONICAL_CONCEPTS.items():
        for v in variants:
            if normalize(v) in norm:
                tags.add(canon)
                break
    return tags

# =========================
# Construcción texto
# =========================
def build_full_text_business(biz: dict) -> str:
    nombre = str(biz.get("nombre", "") or "")
    dc = str(biz.get("descripcion_corta", "") or "")
    dd = str(biz.get("descripcion_detallada", "") or "")
    prod_text_parts = []
    for p in biz.get("productos", []) or []:
        prod_text_parts.append(str((p or {}).get("nombre", "") or ""))
        prod_text_parts.append(str((p or {}).get("descripcion", "") or ""))
    return " ".join([nombre, dc, dd, " ".join(prod_text_parts)])

def build_full_text_product(biz: dict, p: dict) -> str:
    return " ".join([
        str(biz.get("nombre", "") or ""),
        str(biz.get("descripcion_corta", "") or ""),
        str((p or {}).get("nombre", "") or ""),
        str((p or {}).get("descripcion", "") or "")
    ])

# =========================
# Aprendizaje global (cross-negocio)
# =========================

def learn_global_terms(data):

        #El fin: detectar patrones a nivel plataforma, no casos aislados.
        #Aquí es donde se hace la magia para que se autoalimente de conceptos 
        #de negocios nuevos.
        #¿QUÉ HACE PASO A PASO?
                #Aparece “ramen” en muchos negocios
                # El sistema lo detecta como:

                # {
                #   "term": "ramen",
                #   "kind": "label",
                #   "businesses": 9,
                #   "total_mentions": 34
                # }

                # Si cumple:
                # -se auto-promueve
                # -se guarda en label_registry.json
                # Resultado:
                # -Sin escribir reglas
                # -Sin entrenar modelo
                # -Sin tocar código


    """
    Retorna:
    - term_counts: conteo total (tokens y ngrams)
    - term_businesses: set de negocios donde aparece cada término
    - examples: ejemplos (nombre de negocio)
    """
    term_counts = Counter()
    term_businesses = defaultdict(set)
    examples = defaultdict(list)

    for biz in data:
        negocio_id = biz.get("document_id") or ""
        full = build_full_text_business(biz)
        toks = tokenize(full)

        # tokens
        uniq_tokens = set(toks)
        for t in uniq_tokens:
            if is_bad_term(t):
                continue
            term_businesses[t].add(negocio_id)

        term_counts.update([t for t in toks if not is_bad_term(t)])

        # ngrams
        if MAX_NGRAMS >= 2:
            bgs = make_ngrams(toks, 2)
            term_counts.update([bg for bg in bgs if not is_bad_term(bg)])
            for bg in set(bgs):
                if not is_bad_term(bg):
                    term_businesses[bg].add(negocio_id)

        if MAX_NGRAMS >= 3:
            tgs = make_ngrams(toks, 3)
            term_counts.update([tg for tg in tgs if not is_bad_term(tg)])
            for tg in set(tgs):
                if not is_bad_term(tg):
                    term_businesses[tg].add(negocio_id)

        # ejemplos
        biz_name = normalize(biz.get("nombre","") or "")
        for t in list(uniq_tokens)[:20]:
            if len(examples[t]) < 3:
                examples[t].append(biz_name)

    return term_counts, term_businesses, examples

# =========================
# Registro persistente (labels oficiales y promociones)
# =========================
def load_registry():
    if not Path(REGISTRY_PATH).exists():
        return {
            "created_at": datetime.utcnow().isoformat() + "Z",
            "labels_oficiales": sorted(list(LABEL_RULES.keys())),
            "auto_promoted": [],
            "notes": "labels_oficiales se usa para filtros y UI; LABEL_RULES sigue siendo tu clasificador por keywords.",
        }


    return json.loads(Path(REGISTRY_PATH).read_text(encoding="utf-8"))

def save_registry(registry):
    Path(REGISTRY_PATH).write_text(json.dumps(registry, ensure_ascii=False, indent=2), encoding="utf-8")

# =========================
# Extracción local (por negocio) con apoyo global
# =========================
def extract_tags_local(full_text: str):
    toks = tokenize(full_text)
    cnt = Counter(toks)

    if MAX_NGRAMS >= 2:
        cnt.update(make_ngrams(toks, 2))
    if MAX_NGRAMS >= 3:
        cnt.update(make_ngrams(toks, 3))

    # ordenado por frecuencia local
    items = [(w,c) for w,c in cnt.items() if not is_bad_term(w)]
    items.sort(key=lambda x: (-x[1], x[0]))
    return [w for w,_ in items[:TOP_TAGS_PER_BIZ]]

def extract_candidates_from_global(term_counts, term_businesses, registry):
    """
    Produce candidatas "de plataforma" (robustas), no solo del negocio.
    """
    pending = []

    labels_oficiales = set([normalize(x) for x in registry.get("labels_oficiales", [])])
    base_labels = set([normalize(x) for x in LABEL_RULES.keys()])

    for term, total_count in term_counts.items():
        if is_bad_term(term):
            continue

        biz_count = len(term_businesses.get(term, set()))
        if biz_count < MIN_BUSINESSES_FOR_TAG:
            continue

        # Decide si es candidata a label (categoría) por evidencia fuerte
        is_label_candidate = biz_count >= MIN_BUSINESSES_FOR_LABEL

        # Evita promover cosas que ya son label base/oficial
        term_norm = normalize(term)
        already_label = (term_norm in labels_oficiales) or (term_norm in base_labels)

        pending.append({
            "term": term,
            "term_norm": term_norm,
            "total_mentions": int(total_count),
            "businesses": int(biz_count),
            "kind": "label" if (is_label_candidate and not already_label) else "tag",
            "already_label": bool(already_label),
        })

    pending.sort(key=lambda x: (-x["businesses"], -x["total_mentions"], x["term_norm"]))
    return pending

def maybe_auto_promote(pending, registry):
    if not AUTO_PROMOTE_LABEL:
        return registry

    labels_oficiales = set([normalize(x) for x in registry.get("labels_oficiales", [])])

    for item in pending:
        if item["kind"] != "label":
            continue
        if item["already_label"]:
            continue
        if item["businesses"] < AUTO_PROMOTE_MIN_BUSINESSES:
            continue
        if item["total_mentions"] < AUTO_PROMOTE_MIN_TOTAL_MENTIONS:
            continue
        if len(item["term_norm"]) < AUTO_PROMOTE_MIN_LEN:
            continue

        # Promueve como label oficial (para UI/filtros)
        if item["term_norm"] not in labels_oficiales:
            registry.setdefault("labels_oficiales", []).append(item["term_norm"])
            registry.setdefault("auto_promoted", []).append({
                "label": item["term_norm"],
                "businesses": item["businesses"],
                "total_mentions": item["total_mentions"],
                "promoted_at": datetime.utcnow().isoformat() + "Z"
            })
            labels_oficiales.add(item["term_norm"])

    registry["labels_oficiales"] = sorted(list(set(registry["labels_oficiales"])))
    return registry

                    #El fin de labels_oficiales es separar la clasificación 
                    #estable de descubrimiento experimental:
                        # 🔹 Qué hace exactamente:
                        #     -Las LABEL_RULES = reglas duras (tacos, pizza, etc.)
                        #     -labels_oficiales = nuevo vocabulario confiable
                        #     Ej: sushi, ramen, poke, vegan

                        #    Una vez que algo entra aquí:
                        #     -ya se usa en clasificación
                        #     -ya se usa en metadata
                        #     -ya se usa en búsqueda / filtros / RAG



# =========================
# Clasificación por reglas + tags/canónicos + labels oficiales
# =========================
def generate_labels_tags_for_business(biz: dict, registry):
    full_text = build_full_text_business(biz)
    norm_full = normalize(full_text)

    labels = set()
    # 1) labels por reglas base
    for label, kws in LABEL_RULES.items():
        for kw in kws:
            if normalize(kw) in norm_full:
                labels.add(label)
                break

    # 2) tags (canónicos + locales)
    tags = set()
    tags |= apply_canonical_tags(full_text)
    tags |= set(extract_tags_local(full_text))

    # 3) “labels_oficiales” detectados por presencia textual (suave)
    #    (Sirve para cuando auto-promueves algo como "sushi", "ramen", etc.)
    for lbl in registry.get("labels_oficiales", []):
        if not lbl:
            continue
        if normalize(lbl) in norm_full:
            labels.add(normalize(lbl))

    # boost de comida corrida
    if "comida corrida" in tags:
        labels.add("comida_casera")

    return sorted(labels), sorted(tags)

def join_str(values):
    if not values:
        return ""
    return "|".join([normalize(str(v)) for v in values])

# =========================
# MAIN
# =========================
def main():
    data = json.loads(Path(INPUT_DATOS).read_text(encoding="utf-8"))

    # 1) Aprendizaje global (cross-negocio)
    term_counts, term_businesses, examples = learn_global_terms(data)

    # 2) Registro persistente
    registry = load_registry()

    # 3) Pendientes globales y auto-promoción (opcional)
    pending = extract_candidates_from_global(term_counts, term_businesses, registry)
    registry = maybe_auto_promote(pending, registry)
    save_registry(registry)

    # 4) Guardar pendientes con ejemplos
    # añade ejemplos (máximo 3)
    for it in pending[:500]:
        ex = examples.get(it["term"], [])[:3]
        it["examples_businesses"] = ex

    Path(PENDING_PATH).write_text(
        json.dumps({
            "generated_at": datetime.utcnow().isoformat() + "Z",
            "min_businesses_for_tag": MIN_BUSINESSES_FOR_TAG,
            "min_businesses_for_label": MIN_BUSINESSES_FOR_LABEL,
            "auto_promote_label": AUTO_PROMOTE_LABEL,
            "pending": pending[:500],
        }, ensure_ascii=False, indent=2),
        encoding="utf-8"
    )

    # 5) Generación docs negocio + producto
    docs = []

    for biz in data:
        negocio_id = biz.get("document_id")
        if not negocio_id:
            continue

        labels, tags = generate_labels_tags_for_business(biz, registry)

        is_comida_corrida = ("comida corrida" in tags)
        is_tacos = ("tacos" in labels)
        is_pizza = ("pizza" in labels)
        is_hamburguesas = ("hamburguesas" in labels)

        # ========= Doc NEGOCIO =========
        business_text = build_full_text_business(biz)
        text_negocio = "\n".join([
            "tipo_doc: negocio",
            f"negocio_id: {negocio_id}",
            f"nombre: {biz.get('nombre','')}",
            f"descripcion: {biz.get('descripcion_corta','')} {biz.get('descripcion_detallada','')}",
            f"direccion: {biz.get('direccion','')}",
            f"horarios: {(biz.get('horarios') or {}).get('dias','')} {(biz.get('horarios') or {}).get('apertura','')} {(biz.get('horarios') or {}).get('cierre','')}",
            f"labels: {', '.join(labels) if labels else 'ninguno'}",
            f"tags: {', '.join(tags) if tags else 'ninguno'}",
            f"productos: {normalize(business_text)}"
        ])

        docs.append({
            "id": f"{negocio_id}",
            "text": text_negocio,
            "metadata": {
                "tipo_doc": "negocio",
                "negocio_id": negocio_id,
                "nombre": biz.get("nombre"),
                "direccion": biz.get("direccion"),
                "dias": (biz.get("horarios") or {}).get("dias"),
                "apertura": (biz.get("horarios") or {}).get("apertura"),
                "cierre": (biz.get("horarios") or {}).get("cierre"),

                "labels": labels,
                "tags": tags,

                "labels_str": join_str(labels),
                "tags_str": join_str(tags),

                "is_comida_corrida": bool(is_comida_corrida),
                "is_tacos": bool(is_tacos),
                "is_pizza": bool(is_pizza),
                "is_hamburguesas": bool(is_hamburguesas),

                # info para debug/observabilidad
                "labels_oficiales_count": len(registry.get("labels_oficiales", [])),
            }
        })

        # ========= Docs PRODUCTO =========
        for idx, p in enumerate(biz.get("productos", []) or []):
            prod_name = (p or {}).get("nombre")
            prod_desc = (p or {}).get("descripcion")
            prod_price = (p or {}).get("precio")
            if not (prod_name or prod_desc):
                continue

            product_id = f"{negocio_id}::producto_{idx+1}"
            product_text_raw = build_full_text_product(biz, p)

            text_producto = "\n".join([
                "tipo_doc: producto",
                f"negocio_id: {negocio_id}",
                f"nombre_negocio: {biz.get('nombre','')}",
                f"producto_id: {product_id}",
                f"producto: {prod_name or ''}",
                f"descripcion_producto: {prod_desc or ''}",
                f"precio: {prod_price or ''}",
                f"labels: {', '.join(labels) if labels else 'ninguno'}",
                f"tags: {', '.join(tags) if tags else 'ninguno'}",
                f"texto: {normalize(product_text_raw)}"
            ])

            docs.append({
                "id": product_id,
                "text": text_producto,
                "metadata": {
                    "tipo_doc": "producto",
                    "negocio_id": negocio_id,
                    "nombre_negocio": biz.get("nombre"),
                    "producto_id": product_id,
                    "producto_nombre": prod_name,
                    "producto_precio": prod_price,

                    "labels": labels,
                    "tags": tags,

                    "labels_str": join_str(labels),
                    "tags_str": join_str(tags),

                    "is_comida_corrida": bool(is_comida_corrida),
                    "is_tacos": bool(is_tacos),
                    "is_pizza": bool(is_pizza),
                    "is_hamburguesas": bool(is_hamburguesas),
                }
            })

    Path(OUTPUT_DOCS_RAG).write_text(
        json.dumps(docs, ensure_ascii=False, indent=2),
        encoding="utf-8"
    )

    print(f"OK -> generado {OUTPUT_DOCS_RAG} con {len(docs)} docs (negocio + productos).")
    print(f"OK -> generado {PENDING_PATH} (pendientes globales).")
    print(f"OK -> generado {REGISTRY_PATH} (registro persistente).")

if __name__ == "__main__":
    main()

C:\Users\ulise\AppData\Local\Temp\ipykernel_17460\1714667455.py:476: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "generated_at": datetime.utcnow().isoformat() + "Z",


OK -> generado docs_rag.json con 570 docs (negocio + productos).
OK -> generado labels_pendientes.json (pendientes globales).
OK -> generado label_registry.json (registro persistente).


In [73]:
############################################################################3

##### Texto listo para embeddings (Convierte cada negocio en un texto listo para embeddings.) Es aprendizaje automatico de docs_rag.json, en pocas palabras es ML
